In [49]:
from copy import deepcopy
def generate_vaddpd_instructions(LDX, MINUS=128):
    X = (LDX-MINUS) // 8
    instructions = []
    
    # 最初に必要なレジスターのリストを作成
    reg_list = [f'zmm{i}' for i in range(1, X+1)]
    n_1st_reg = len(reg_list)
#     print("n_1st_reg: ", n_1st_reg)
    if (n_1st_reg==0):
        return f'            zmm0 = _mm512_add_pd(zmm0, zmm1);'
    if n_1st_reg == 1:
        instructions.append(f'            zmm0 = _mm512_add_pd(zmm0, {reg_list[0]});')
    else:
        reg_list = reg_list if len(reg_list)%2==0 else reg_list[:-1]
#         print("BASE: ", reg_list, n_1st_reg)

        # 最初の加算
        reg_list_ = [] # 加算結果が格納されたレジスタを格納する
        for i in range(1, len(reg_list)+1, 2):
            instructions.append(f'            zmm{i} = _mm512_add_pd(zmm{i}, zmm{i+1});')
            reg_list_.append(f'zmm{i}')
#         print("First: ", reg_list_)

        # 2回目以降の加算
        while len(reg_list_)>1:
            instructions.append("")
            reg_list__ = [] # 加算結果が格納されたレジスタを格納する
    #         print(f'reg_list_: {reg_list_}')
            for i in range(0, len(reg_list_)//2*2, 2):
                instructions.append(f'            {reg_list_[i]} = _mm512_add_pd({reg_list_[i]}, {reg_list_[i+1]});')
                reg_list__.append(f'{reg_list_[i]}')
            if len(reg_list_) % 2 == 1:
                instructions.append(f'            {reg_list_[-3]} = _mm512_add_pd({reg_list_[-1]}, {reg_list_[-3]});')

            reg_list_ = deepcopy(reg_list__)
            if len(reg_list_)==1: break
#             print("Second: ", reg_list_)

        # 最後
        instructions.append("")
        if (n_1st_reg%2==1): instructions.append(f'            zmm0 = _mm512_add_pd(zmm0, zmm{n_1st_reg});')
        instructions.append(f'            zmm0 = _mm512_add_pd(zmm0, {reg_list_[0]});')

    return "\n".join(instructions)
# print(generate_vaddpd_instructions(144))
generate_vaddpd_instructions(256, MINUS=256)



'            zmm0 = _mm512_add_pd(zmm0, zmm1);'

In [50]:
def generate_remaining_part_vaddpd_instructions(LDX_MOD, LDX):
    if (LDX - LDX_MOD == 8):
        return f'            zmm0 = _mm512_add_pd(zmm0, zmm1);'
    
    X = (LDX-LDX_MOD) // 8
    instructions = []
    
    # 最初に必要なレジスターのリストを作成
    reg_list = [f'zmm{i}' for i in range(1, X+1)]
    n_1st_reg = len(reg_list)
#     print("n_1st_reg: ", n_1st_reg)
    if (n_1st_reg==0):
        return f'            zmm0 = _mm512_add_pd(zmm0, zmm1);'
    if n_1st_reg == 1:
        instructions.append(f'            zmm0 = _mm512_add_pd(zmm0, {reg_list[0]});')
    else:
        reg_list = reg_list if len(reg_list)%2==0 else reg_list[:-1]
#         print("BASE: ", reg_list, n_1st_reg)

        # 最初の加算
        reg_list_ = [] # 加算結果が格納されたレジスタを格納する
        for i in range(1, len(reg_list)+1, 2):
            instructions.append(f'            zmm{i} = _mm512_add_pd(zmm{i}, zmm{i+1});')
            reg_list_.append(f'zmm{i}')
#         print("First: ", reg_list_)

        # 2回目以降の加算
        while len(reg_list_)>1:
            instructions.append("")
            reg_list__ = [] # 加算結果が格納されたレジスタを格納する
    #         print(f'reg_list_: {reg_list_}')
            for i in range(0, len(reg_list_)//2*2, 2):
                instructions.append(f'            {reg_list_[i]} = _mm512_add_pd({reg_list_[i]}, {reg_list_[i+1]});')
                reg_list__.append(f'{reg_list_[i]}')
            if len(reg_list_) % 2 == 1:
                instructions.append(f'            {reg_list_[-3]} = _mm512_add_pd({reg_list_[-1]}, {reg_list_[-3]});')

            reg_list_ = deepcopy(reg_list__)
            if len(reg_list_)==1: break
#             print("Second: ", reg_list_)

        # 最後
        instructions.append("")
        if (n_1st_reg%2==1): instructions.append(f'            zmm0 = _mm512_add_pd(zmm0, zmm{n_1st_reg});')
        instructions.append(f'            zmm0 = _mm512_add_pd(zmm0, {reg_list_[0]});')

    return "\n".join(instructions)

generate_remaining_part_vaddpd_instructions(248, 254)

'            zmm0 = _mm512_add_pd(zmm0, zmm1);'

In [51]:
def remain_part(LDX):
    rem = LDX % 8
    IDX = LDX - rem
    if rem == 0:
        return ""
    elif rem == 2:
        return f'''
            // Load data from x
            xmm29 = _mm_load_pd(&x[{IDX}]);

            // Load data from a_t
            xmm14 = _mm_load_pd(&a_t[{IDX}]);

            // Perform multiplication
            xmm29 = _mm_mul_pd(xmm29, xmm14);            
        '''
    elif rem == 4:
        return f'''
            // Load data from x
            ymm30 = _mm256_loadu_pd(&x[{IDX}]);

            // Load data from a_t
            ymm15 = _mm256_loadu_pd(&a_t[{IDX}]);

            // Perform multiplication
            ymm30 = _mm256_mul_pd(ymm30, ymm15);
        '''
    elif rem == 6:
        return f'''
            // Load data from x
            ymm30 = _mm256_loadu_pd(&x[{IDX}]);
            xmm29 = _mm_load_pd(&x[{IDX+4}]);

            // Load data from a_t
            ymm15 = _mm256_loadu_pd(&a_t[{IDX}]);
            xmm14 = _mm_load_pd(&a_t[{IDX+4}]);
            
            
            
            
            

            // Perform multiplication
            ymm30 = _mm256_mul_pd(ymm30, ymm15);
            xmm29 = _mm_mul_pd(xmm29, xmm14);            
        '''
    
def result_part(LDX):
    rem = LDX % 8
    if rem == 0:
        return f"            y[i] = sum_zmm_elements(zmm0); a_t += {LDX};"
    elif rem == 2:
        return f"            y[i] = sum_zmm_elements(zmm0) + sum_xmm_elements(xmm29); a_t += {LDX};"
    elif rem == 4:
        return f"            y[i] = sum_zmm_elements(zmm0) + sum_ymm_elements(ymm30); a_t += {LDX};"
    elif rem == 6:
        return f"            y[i] = sum_zmm_elements(zmm0) + sum_ymm_elements(ymm30) + sum_xmm_elements(xmm29); a_t += {LDX};"

In [52]:
LDX = 144

SHO = LDX - 128
N_8 = SHO // 8
REM = SHO % 8

LOAD_X = "\n".join([f'            zmm{i} = _mm512_loadu_pd(&x[{i*8}]);' for i in range(16, N_8+16, 1)])
LOAD_A = "\n".join([f'            zmm{i} = _mm512_loadu_pd(&a_t[{i*8+120}]);' for i in range(1, N_8+1)])

MUL_XA =  "\n".join([f'            zmm{i}  = _mm512_mul_pd(zmm{i}, zmm{i+15});' for i in range(1, N_8+1)])

func_str = f'''
void mydgemv_a_z_{LDX}_(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
    __m512d zmm0, zmm1, zmm2, zmm3, zmm4, zmm5, zmm6, zmm7, zmm8, zmm9, zmm10, zmm11, zmm12, zmm13, zmm14, zmm15;
    __m512d zmm16, zmm17, zmm18, zmm19, zmm20, zmm21, zmm22, zmm23, zmm24, zmm25, zmm26, zmm27, zmm28, zmm29, zmm30, zmm31;
    __m256d ymm30, ymm15;
    __m128d xmm29, xmm14;
    double result;

    for (int64_t i = 0; i < lda; ++i) {{
        zmm16 = _mm512_loadu_pd(&x[0 * 8]);
        zmm17 = _mm512_loadu_pd(&x[1 * 8]);
        zmm18 = _mm512_loadu_pd(&x[2 * 8]);
        zmm19 = _mm512_loadu_pd(&x[3 * 8]);
        zmm20 = _mm512_loadu_pd(&x[4 * 8]);
        zmm21 = _mm512_loadu_pd(&x[5 * 8]);
        zmm22 = _mm512_loadu_pd(&x[6 * 8]);
        zmm23 = _mm512_loadu_pd(&x[7 * 8]);
        zmm24 = _mm512_loadu_pd(&x[8 * 8]);
        zmm25 = _mm512_loadu_pd(&x[9 * 8]);
        zmm26 = _mm512_loadu_pd(&x[10 * 8]);
        zmm27 = _mm512_loadu_pd(&x[11 * 8]);
        zmm28 = _mm512_loadu_pd(&x[12 * 8]);
        zmm29 = _mm512_loadu_pd(&x[13 * 8]);
        zmm30 = _mm512_loadu_pd(&x[14 * 8]);
        zmm31 = _mm512_loadu_pd(&x[15 * 8]);

        zmm0 = _mm512_loadu_pd(&a_t[0 * 8]);
        zmm1 = _mm512_loadu_pd(&a_t[1 * 8]);
        zmm2 = _mm512_loadu_pd(&a_t[2 * 8]);
        zmm3 = _mm512_loadu_pd(&a_t[3 * 8]);
        zmm4 = _mm512_loadu_pd(&a_t[4 * 8]);
        zmm5 = _mm512_loadu_pd(&a_t[5 * 8]);
        zmm6 = _mm512_loadu_pd(&a_t[6 * 8]);
        zmm7 = _mm512_loadu_pd(&a_t[7 * 8]);
        zmm8 = _mm512_loadu_pd(&a_t[8 * 8]);
        zmm9 = _mm512_loadu_pd(&a_t[9 * 8]);
        zmm10 = _mm512_loadu_pd(&a_t[10 * 8]);
        zmm11 = _mm512_loadu_pd(&a_t[11 * 8]);
        zmm12 = _mm512_loadu_pd(&a_t[12 * 8]);
        zmm13 = _mm512_loadu_pd(&a_t[13 * 8]);
        zmm14 = _mm512_loadu_pd(&a_t[14 * 8]);
        zmm15 = _mm512_loadu_pd(&a_t[15 * 8]);



        // Perform multiplication
        zmm0 = _mm512_mul_pd(zmm0, zmm16);
        zmm1 = _mm512_mul_pd(zmm1, zmm17);
        zmm2 = _mm512_mul_pd(zmm2, zmm18);
        zmm3 = _mm512_mul_pd(zmm3, zmm19);
        zmm4 = _mm512_mul_pd(zmm4, zmm20);
        zmm5 = _mm512_mul_pd(zmm5, zmm21);
        zmm6 = _mm512_mul_pd(zmm6, zmm22);
        zmm7 = _mm512_mul_pd(zmm7, zmm23);
        zmm8 = _mm512_mul_pd(zmm8, zmm24);
        zmm9 = _mm512_mul_pd(zmm9, zmm25);
        zmm10 = _mm512_mul_pd(zmm10, zmm26);
        zmm11 = _mm512_mul_pd(zmm11, zmm27);
        zmm12 = _mm512_mul_pd(zmm12, zmm28);
        zmm13 = _mm512_mul_pd(zmm13, zmm29);
        zmm14 = _mm512_mul_pd(zmm14, zmm30);
        zmm15 = _mm512_mul_pd(zmm15, zmm31);

        // Perform addition
        zmm1 = _mm512_add_pd(zmm0, zmm1);
        zmm3 = _mm512_add_pd(zmm2, zmm3);
        zmm5 = _mm512_add_pd(zmm4, zmm5);
        zmm7 = _mm512_add_pd(zmm6, zmm7);
        zmm9 = _mm512_add_pd(zmm8, zmm9);
        zmm11 = _mm512_add_pd(zmm10, zmm11);
        zmm13 = _mm512_add_pd(zmm12, zmm13);
        zmm15 = _mm512_add_pd(zmm14, zmm15);

        zmm3 = _mm512_add_pd(zmm1, zmm3);
        zmm7 = _mm512_add_pd(zmm5, zmm7);
        zmm11 = _mm512_add_pd(zmm9, zmm11);
        zmm15 = _mm512_add_pd(zmm13, zmm15);

        zmm7 = _mm512_add_pd(zmm3, zmm7);
        zmm15 = _mm512_add_pd(zmm11, zmm15);

        zmm0 = _mm512_add_pd(zmm7, zmm15);


        // ----------------------------------------------------------------------------
{LOAD_X}

{LOAD_A}

{MUL_XA}

{generate_vaddpd_instructions(LDX)}

{remain_part(LDX)}

{result_part(LDX)}
    }} 
}}
'''
print(func_str)


void mydgemv_a_z_144_(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){
    __m512d zmm0, zmm1, zmm2, zmm3, zmm4, zmm5, zmm6, zmm7, zmm8, zmm9, zmm10, zmm11, zmm12, zmm13, zmm14, zmm15;
    __m512d zmm16, zmm17, zmm18, zmm19, zmm20, zmm21, zmm22, zmm23, zmm24, zmm25, zmm26, zmm27, zmm28, zmm29, zmm30, zmm31;
    __m256d ymm30, ymm15;
    __m128d xmm29, xmm14;
    double result;

    for (int64_t i = 0; i < lda; ++i) {
        zmm16 = _mm512_loadu_pd(&x[0 * 8]);
        zmm17 = _mm512_loadu_pd(&x[1 * 8]);
        zmm18 = _mm512_loadu_pd(&x[2 * 8]);
        zmm19 = _mm512_loadu_pd(&x[3 * 8]);
        zmm20 = _mm512_loadu_pd(&x[4 * 8]);
        zmm21 = _mm512_loadu_pd(&x[5 * 8]);
        zmm22 = _mm512_loadu_pd(&x[6 * 8]);
        zmm23 = _mm512_loadu_pd(&x[7 * 8]);
        zmm24 = _mm512_loadu_pd(&x[8 * 8]);
        zmm25 = _mm512_loadu_pd(&x[9 * 8]);
        zmm26 = _mm512_loadu_pd(&x[10 * 8]);
        zmm27 = _mm512_loadu_pd(&x[11 * 8]);
    

In [53]:
MAIN = """
                // 0-128 ----------------------------------------------------------------------------
                zmm16 = _mm512_loadu_pd(&x[0 * 8]);
                zmm17 = _mm512_loadu_pd(&x[1 * 8]);
                zmm18 = _mm512_loadu_pd(&x[2 * 8]);
                zmm19 = _mm512_loadu_pd(&x[3 * 8]);
                zmm20 = _mm512_loadu_pd(&x[4 * 8]);
                zmm21 = _mm512_loadu_pd(&x[5 * 8]);
                zmm22 = _mm512_loadu_pd(&x[6 * 8]);
                zmm23 = _mm512_loadu_pd(&x[7 * 8]);
                zmm24 = _mm512_loadu_pd(&x[8 * 8]);
                zmm25 = _mm512_loadu_pd(&x[9 * 8]);
                zmm26 = _mm512_loadu_pd(&x[10 * 8]);
                zmm27 = _mm512_loadu_pd(&x[11 * 8]);
                zmm28 = _mm512_loadu_pd(&x[12 * 8]);
                zmm29 = _mm512_loadu_pd(&x[13 * 8]);
                zmm30 = _mm512_loadu_pd(&x[14 * 8]);
                zmm31 = _mm512_loadu_pd(&x[15 * 8]);

                zmm0 = _mm512_loadu_pd(&a_t[0 * 8]);
                zmm1 = _mm512_loadu_pd(&a_t[1 * 8]);
                zmm2 = _mm512_loadu_pd(&a_t[2 * 8]);
                zmm3 = _mm512_loadu_pd(&a_t[3 * 8]);
                zmm4 = _mm512_loadu_pd(&a_t[4 * 8]);
                zmm5 = _mm512_loadu_pd(&a_t[5 * 8]);
                zmm6 = _mm512_loadu_pd(&a_t[6 * 8]);
                zmm7 = _mm512_loadu_pd(&a_t[7 * 8]);
                zmm8 = _mm512_loadu_pd(&a_t[8 * 8]);
                zmm9 = _mm512_loadu_pd(&a_t[9 * 8]);
                zmm10 = _mm512_loadu_pd(&a_t[10 * 8]);
                zmm11 = _mm512_loadu_pd(&a_t[11 * 8]);
                zmm12 = _mm512_loadu_pd(&a_t[12 * 8]);
                zmm13 = _mm512_loadu_pd(&a_t[13 * 8]);
                zmm14 = _mm512_loadu_pd(&a_t[14 * 8]);
                zmm15 = _mm512_loadu_pd(&a_t[15 * 8]);



                // Perform multiplication
                zmm0 = _mm512_mul_pd(zmm0, zmm16);
                zmm1 = _mm512_mul_pd(zmm1, zmm17);
                zmm2 = _mm512_mul_pd(zmm2, zmm18);
                zmm3 = _mm512_mul_pd(zmm3, zmm19);
                zmm4 = _mm512_mul_pd(zmm4, zmm20);
                zmm5 = _mm512_mul_pd(zmm5, zmm21);
                zmm6 = _mm512_mul_pd(zmm6, zmm22);
                zmm7 = _mm512_mul_pd(zmm7, zmm23);
                zmm8 = _mm512_mul_pd(zmm8, zmm24);
                zmm9 = _mm512_mul_pd(zmm9, zmm25);
                zmm10 = _mm512_mul_pd(zmm10, zmm26);
                zmm11 = _mm512_mul_pd(zmm11, zmm27);
                zmm12 = _mm512_mul_pd(zmm12, zmm28);
                zmm13 = _mm512_mul_pd(zmm13, zmm29);
                zmm14 = _mm512_mul_pd(zmm14, zmm30);
                zmm15 = _mm512_mul_pd(zmm15, zmm31);

                // Perform addition
                zmm1 = _mm512_add_pd(zmm0, zmm1);
                zmm3 = _mm512_add_pd(zmm2, zmm3);
                zmm5 = _mm512_add_pd(zmm4, zmm5);
                zmm7 = _mm512_add_pd(zmm6, zmm7);
                zmm9 = _mm512_add_pd(zmm8, zmm9);
                zmm11 = _mm512_add_pd(zmm10, zmm11);
                zmm13 = _mm512_add_pd(zmm12, zmm13);
                zmm15 = _mm512_add_pd(zmm14, zmm15);

                zmm3 = _mm512_add_pd(zmm1, zmm3);
                zmm7 = _mm512_add_pd(zmm5, zmm7);
                zmm11 = _mm512_add_pd(zmm9, zmm11);
                zmm15 = _mm512_add_pd(zmm13, zmm15);

                zmm7 = _mm512_add_pd(zmm3, zmm7);
                zmm15 = _mm512_add_pd(zmm11, zmm15);

                zmm0 = _mm512_add_pd(zmm7, zmm15);
"""

In [56]:
# 10 % 2
# 130 - 254まで

with open("./FortLearner_latest_20220502/FortLearner/work/small_gemv/small_dgemv_naive_c_implementation_ver3.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
     
    for i in range(128, 512, 2):
        LDX = i

        SHO = LDX - 128
        N_8 = SHO // 8
        REM = SHO % 8

        LOAD_X = "\n".join([f'            zmm{i} = _mm512_loadu_pd(&x[{i*8}]);' for i in range(16, N_8+16, 1)])
        LOAD_A = "\n".join([f'            zmm{i} = _mm512_loadu_pd(&a_t[{i*8+120}]);' for i in range(1, N_8+1)])

        MUL_XA =  "\n".join([f'            zmm{i}  = _mm512_mul_pd(zmm{i}, zmm{i+15});' for i in range(1, N_8+1)])

        func_str = f'''
        void mydgemv_a_z_{LDX}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
            __m512d zmm0, zmm1, zmm2, zmm3, zmm4, zmm5, zmm6, zmm7, zmm8, zmm9, zmm10, zmm11, zmm12, zmm13, zmm14, zmm15;
            __m512d zmm16, zmm17, zmm18, zmm19, zmm20, zmm21, zmm22, zmm23, zmm24, zmm25, zmm26, zmm27, zmm28, zmm29, zmm30, zmm31;
            __m256d ymm30, ymm15;
            __m128d xmm29, xmm14;
            double result;

            for (int64_t i = 0; i < lda; ++i) {{
{MAIN}

                // ----------------------------------------------------------------------------
        {LOAD_X}

        {LOAD_A}

        {MUL_XA}

        {generate_vaddpd_instructions(LDX)}

        {remain_part(LDX)}

        {result_part(LDX)}
            }} 
        }}
        '''
        p.write(func_str)

In [48]:
# 10 % 2
# 130 - 254まで

with open("code", "w") as p:
    for i in range(280, 1026, 2):
        LDX = i

        SHO = LDX - 128
        N_8 = SHO // 8
        REM = SHO % 8
        
        LDX_MOD128 = LDX - REM
        REMAIN_LDX = LDX - 128
        START_IDX = 128
#         print(LDX, SHO, N_8, REM)


        # 120 
        MAIN_120 = ""
        while (REMAIN_LDX>=120):
            MAIN_120 += f"""
            // {START_IDX}-{START_IDX+120} ----------------------------------------------------------------------------
            zmm16 = _mm512_loadu_pd(&x[{START_IDX + 0 * 8}]);
            zmm17 = _mm512_loadu_pd(&x[{START_IDX + 1 * 8}]);
            zmm18 = _mm512_loadu_pd(&x[{START_IDX + 2 * 8}]);
            zmm19 = _mm512_loadu_pd(&x[{START_IDX + 3 * 8}]);
            zmm20 = _mm512_loadu_pd(&x[{START_IDX + 4 * 8}]);
            zmm21 = _mm512_loadu_pd(&x[{START_IDX + 5 * 8}]);
            zmm22 = _mm512_loadu_pd(&x[{START_IDX + 6 * 8}]);
            zmm23 = _mm512_loadu_pd(&x[{START_IDX + 7 * 8}]);
            zmm24 = _mm512_loadu_pd(&x[{START_IDX + 8 * 8}]);
            zmm25 = _mm512_loadu_pd(&x[{START_IDX + 9 * 8}]);
            zmm26 = _mm512_loadu_pd(&x[{START_IDX + 10 * 8}]);
            zmm27 = _mm512_loadu_pd(&x[{START_IDX + 11 * 8}]);
            zmm28 = _mm512_loadu_pd(&x[{START_IDX + 12 * 8}]);
            zmm29 = _mm512_loadu_pd(&x[{START_IDX + 13 * 8}]);
            zmm30 = _mm512_loadu_pd(&x[{START_IDX + 14 * 8}]);

            zmm1 = _mm512_loadu_pd(&a_t[{START_IDX + 0 * 8}]);
            zmm2 = _mm512_loadu_pd(&a_t[{START_IDX + 1 * 8}]);
            zmm3 = _mm512_loadu_pd(&a_t[{START_IDX + 2 * 8}]);
            zmm4 = _mm512_loadu_pd(&a_t[{START_IDX + 3 * 8}]);
            zmm5 = _mm512_loadu_pd(&a_t[{START_IDX + 4 * 8}]);
            zmm6 = _mm512_loadu_pd(&a_t[{START_IDX + 5 * 8}]);
            zmm7 = _mm512_loadu_pd(&a_t[{START_IDX + 6 * 8}]);
            zmm8 = _mm512_loadu_pd(&a_t[{START_IDX + 7 * 8}]);
            zmm9 = _mm512_loadu_pd(&a_t[{START_IDX + 8 * 8}]);
            zmm10 = _mm512_loadu_pd(&a_t[{START_IDX + 9 * 8}]);
            zmm11 = _mm512_loadu_pd(&a_t[{START_IDX + 10 * 8}]);
            zmm12 = _mm512_loadu_pd(&a_t[{START_IDX + 11 * 8}]);
            zmm13 = _mm512_loadu_pd(&a_t[{START_IDX + 12 * 8}]);
            zmm14 = _mm512_loadu_pd(&a_t[{START_IDX + 13 * 8}]);
            zmm15 = _mm512_loadu_pd(&a_t[{START_IDX + 14 * 8}]);

            zmm1  = _mm512_mul_pd(zmm1, zmm16);
            zmm2  = _mm512_mul_pd(zmm2, zmm17);
            zmm3  = _mm512_mul_pd(zmm3, zmm18);
            zmm4  = _mm512_mul_pd(zmm4, zmm19);
            zmm5  = _mm512_mul_pd(zmm5, zmm20);
            zmm6  = _mm512_mul_pd(zmm6, zmm21);
            zmm7  = _mm512_mul_pd(zmm7, zmm22);
            zmm8  = _mm512_mul_pd(zmm8, zmm23);
            zmm9  = _mm512_mul_pd(zmm9, zmm24);
            zmm10  = _mm512_mul_pd(zmm10, zmm25);
            zmm11  = _mm512_mul_pd(zmm11, zmm26);
            zmm12  = _mm512_mul_pd(zmm12, zmm27);
            zmm13  = _mm512_mul_pd(zmm13, zmm28);
            zmm14  = _mm512_mul_pd(zmm14, zmm29);
            zmm15  = _mm512_mul_pd(zmm15, zmm30);

            zmm1 = _mm512_add_pd(zmm1, zmm2);
            zmm3 = _mm512_add_pd(zmm3, zmm4);
            zmm5 = _mm512_add_pd(zmm5, zmm6);
            zmm7 = _mm512_add_pd(zmm7, zmm8);
            zmm9 = _mm512_add_pd(zmm9, zmm10);
            zmm11 = _mm512_add_pd(zmm11, zmm12);
            zmm13 = _mm512_add_pd(zmm13, zmm14);

            zmm1 = _mm512_add_pd(zmm1, zmm3);
            zmm5 = _mm512_add_pd(zmm5, zmm7);
            zmm9 = _mm512_add_pd(zmm9, zmm11);
            zmm9 = _mm512_add_pd(zmm13, zmm9);

            zmm1 = _mm512_add_pd(zmm1, zmm5);
            zmm1 = _mm512_add_pd(zmm9, zmm1);

            zmm0 = _mm512_add_pd(zmm0, zmm15);
            zmm0 = _mm512_add_pd(zmm0, zmm1);
            """
            REMAIN_LDX -= 120
            START_IDX += 120
        
#         print(START_IDX, REMAIN_LDX)
        
        N_8 = REMAIN_LDX // 8        
        LOAD_X = "\n".join([f'            zmm{31-i} = _mm512_loadu_pd(&x[{START_IDX+i*8}]);' for i in range(0, N_8, 1)])
        LOAD_A = "\n".join([f'            zmm{i+1}  = _mm512_loadu_pd(&a_t[{START_IDX+i*8}]);' for i in range(0, N_8,1)])
        MUL_XA = "\n".join([f'            zmm{i+1}  = _mm512_mul_pd(zmm{i+1}, zmm{31-i});' for i in range(0, N_8, 1)])
#         print(LOAD_X)
#         print("")
#         print(LOAD_A)
#         print("")
#         print(MUL_XA)
        func_str = f'''
        void mydgemv_a_z_{LDX}_(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
            __m512d zmm0, zmm1, zmm2, zmm3, zmm4, zmm5, zmm6, zmm7, zmm8, zmm9, zmm10, zmm11, zmm12, zmm13, zmm14, zmm15;
            __m512d zmm16, zmm17, zmm18, zmm19, zmm20, zmm21, zmm22, zmm23, zmm24, zmm25, zmm26, zmm27, zmm28, zmm29, zmm30, zmm31;
            __m256d ymm30, ymm15;
            __m128d xmm29, xmm14;
            double result;

            for (int64_t i = 0; i < lda; ++i) {{
{MAIN}

{MAIN_120}
        
            // {START_IDX}
            // LOAD_X
{LOAD_X}

            // LOAD_A
{LOAD_A}

            // MUL_XA
{MUL_XA}

            // generate_vaddpd_instructions
{generate_remaining_part_vaddpd_instructions(START_IDX, LDX_MOD128)}

            // remain_part
{remain_part(LDX)}

            // result_part
{result_part(LDX)}
            }} 
        }}
        '''
        p.write(func_str)
#         print(func_str)
#         break

NameError: name 'generate_remaining_part_vaddpd_instructions' is not defined

In [42]:
generate_vaddpd_instructions(256, MINUS=256)

NameError: name 'generate_vaddpd_instructions' is not defined

In [ ]:
last_idxs = (0, 2)
for lll in range(4, 33, 1):
    ldx = lll
    declare_x = "double " + ", ".join([f"r{ld:0=2}" for ld in range(ldx)])

    mul_add = [
        f"""
            r{ld+0:0=2} = a_t[{ld}+base_idx];
            r{ld+0:0=2} = x[{ld}]*r{ld+0:0=2};    
        """        
        if ( (ldx % 2 == 1) & (ld==ldx-1) ) else
        f"""
            r{ld+0:0=2} = a_t[{ld}+base_idx];
            r{ld+1:0=2} = a_t[{ld+1}+base_idx]; 
            r{ld+0:0=2} = x[{ld}]*r{ld+0:0=2} + x[{ld+1}]*r{ld+1:0=2};    
        """
        for ld in range(0,ldx,2)
    ]
    mul_add = "".join(mul_add)

    start_step = 4
    accm = ""
    step_size = start_step
    while (step_size < ldx):
        accm += "".join(
            [f"""
            r{ld:0=2} = r{ld:0=2} + r{ld+step_size//2:0=2};
            r{ld+step_size:0=2} = r{ld+step_size:0=2} + r{ld+step_size+step_size//2:0=2};
            """
            if (ld+step_size+step_size//2<ldx) else
            f"""
            r{ld:0=2} = r{ld:0=2} + r{ld+step_size//2:0=2};
            """
            for ld in range(0, ldx, step_size*2)])

        last_idxs = [(ld, ld+step_size) for ld in range(0, ldx, step_size*2)][0]
        step_size *= 2
    accm += f"""
            r{last_idxs[0]:0=2} = r{last_idxs[0]:0=2} + r{last_idxs[1]:0=2};
    """
    # print(accm)

    base = f"""
    void mydgemv_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
        {declare_x};

        int64_t base_idx;
        for (int i=0; i<lda; i++){{
            base_idx = i*ldx;
            {mul_add}
            {accm}
            y[i]=r{last_idxs[0]:0=2};
        }}
    }}
    """
    print(base)

In [ ]:
last_idxs = (0, 2)

for lll in range(4, 34, 1):
    ldx = lll
    declare_x = "double " + ", ".join([f"r{ld:0=2}" for ld in range(ldx)])

    mul_add = [
        f"""
        r{ld+0:0=2} = r[{ld}];
        r{ld+0:0=2} = x[{ld}]*r{ld+0:0=2};    
        """        
        if ( (ldx % 2 == 1) & (ld==ldx-1) ) else
        f"""
        r{ld+0:0=2} = r[{ld}];
        r{ld+1:0=2} = r[{ld+1}]; 
        r{ld+0:0=2} = x[{ld}]*r{ld+0:0=2} + x[{ld+1}]*r{ld+1:0=2};    
        """
        for ld in range(0,ldx,2)
    ]
    mul_add = "".join(mul_add)

    start_step = 4
    accm = ""
    step_size = start_step
    while (step_size < ldx):
        accm += "".join(
            [f"""
        r{ld:0=2} = r{ld:0=2} + r{ld+step_size//2:0=2};
        r{ld+step_size:0=2} = r{ld+step_size:0=2} + r{ld+step_size+step_size//2:0=2};
            """
            if (ld+step_size+step_size//2<ldx) else
            f"""
        r{ld:0=2} = r{ld:0=2} + r{ld+step_size//2:0=2};
            """
            for ld in range(0, ldx, step_size*2)])

        last_idxs = [(ld, ld+step_size) for ld in range(0, ldx, step_size*2)][0]
        step_size *= 2
    accm += f"""
        r{last_idxs[0]:0=2} = r{last_idxs[0]:0=2} + r{last_idxs[1]:0=2};
    """
    # print(accm)

    base = f"""
    double mydgemv_kernel_{ldx}(double r[], double x[]){{
        {declare_x};

        int64_t base_idx;
        {mul_add}
        {accm}
        
        return r00;
    }}
    """
    print(base)
#     break

In [2]:
# ldx = 3
# declare_x = "double " + ", ".join([f"r{ld:0=2} = x[{ld}]" for ld in range(ldx)])
# declare_a = "double " + ", ".join([f"r{ld+ldx:0=2}" for ld in range(ldx)])

# mul_add = [
#     f"""
#         r{ld+0+ldx:0=2} = a_t[{ld}+base_idx];
#         r{ld+1+ldx:0=2} = a_t[{ld+1}+base_idx]; 
#         r{ld+0+ldx:0=2} = r{ld:0=2}*r{ld+0+ldx:0=2} + r{ld+1:0=2}*r{ld+1+ldx:0=2};    
#     """
#     for ld in range(0,ldx,2)
# ]
# mul_add = "".join(mul_add)

# start_step = 2
# accm = ""
# step_size = start_step
# while (step_size < ldx):
#     accm += "".join(
#         [f"""
#         r{ld} = r{ld} + r{ld+step_size//2};
#         r{ld+step_size} = r{ld+step_size} + r{ld+step_size+step_size//2};
#         """
#         if (ld+step_size+step_size//2<ldx*2) else
#         f"""
#         r{ld} = r{ld} + r{ld+step_size//2};
#         """
#         for ld in range(ldx, ldx*2, step_size*2)])
    
#     last_idxs = [(ld, ld+step_size) for ld in range(ldx, ldx*2, step_size*2)][0]
#     step_size *= 2
# accm += f"""
#         r{last_idxs[0]} = r{last_idxs[0]} + r{last_idxs[1]};
# """
# # print(accm)

# base = f"""
# void mydgemv_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
#     {declare_x};
#     {declare_a};
    
#     int64_t base_idx;
#     for (int i=0; i<lda; i++){{
#         base_idx = i*ldx;
#         {mul_add}
#         {accm}
#         y[i]=r{last_idxs[0]};
#     }}
# }}
# """
# print(base)

In [3]:
224 

224

In [4]:
last_idxs

NameError: name 'last_idxs' is not defined

In [117]:
for i in range(280, 1026, 2):
    print(f"if (ldx == {i}) subPtr_z => mydgemv_a_z_{i}_")

if (ldx == 280) subPtr_z => mydgemv_a_z_280_
if (ldx == 282) subPtr_z => mydgemv_a_z_282_
if (ldx == 284) subPtr_z => mydgemv_a_z_284_
if (ldx == 286) subPtr_z => mydgemv_a_z_286_
if (ldx == 288) subPtr_z => mydgemv_a_z_288_
if (ldx == 290) subPtr_z => mydgemv_a_z_290_
if (ldx == 292) subPtr_z => mydgemv_a_z_292_
if (ldx == 294) subPtr_z => mydgemv_a_z_294_
if (ldx == 296) subPtr_z => mydgemv_a_z_296_
if (ldx == 298) subPtr_z => mydgemv_a_z_298_
if (ldx == 300) subPtr_z => mydgemv_a_z_300_
if (ldx == 302) subPtr_z => mydgemv_a_z_302_
if (ldx == 304) subPtr_z => mydgemv_a_z_304_
if (ldx == 306) subPtr_z => mydgemv_a_z_306_
if (ldx == 308) subPtr_z => mydgemv_a_z_308_
if (ldx == 310) subPtr_z => mydgemv_a_z_310_
if (ldx == 312) subPtr_z => mydgemv_a_z_312_
if (ldx == 314) subPtr_z => mydgemv_a_z_314_
if (ldx == 316) subPtr_z => mydgemv_a_z_316_
if (ldx == 318) subPtr_z => mydgemv_a_z_318_
if (ldx == 320) subPtr_z => mydgemv_a_z_320_
if (ldx == 322) subPtr_z => mydgemv_a_z_322_
if (ldx ==

In [118]:
for i in range(280, 1026, 2):
    print(f"""
subroutine mydgemv_a_z_{i}_(a_t, x, y, lda, ldx, ldy, shift) Bind(C,Name='mydgemv_a_z_{i}_')
    Import
    Integer(c_int64_t),Value :: lda, ldx, ldy, shift
    Real(c_double),Intent(In) :: a_t(ldx, lda), x(ldx), y(ldy)
end subroutine    """)


subroutine mydgemv_a_z_280_(a_t, x, y, lda, ldx, ldy, shift) Bind(C,Name='mydgemv_a_z_280_')
    Import
    Integer(c_int64_t),Value :: lda, ldx, ldy, shift
    Real(c_double),Intent(In) :: a_t(ldx, lda), x(ldx), y(ldy)
end subroutine    

subroutine mydgemv_a_z_282_(a_t, x, y, lda, ldx, ldy, shift) Bind(C,Name='mydgemv_a_z_282_')
    Import
    Integer(c_int64_t),Value :: lda, ldx, ldy, shift
    Real(c_double),Intent(In) :: a_t(ldx, lda), x(ldx), y(ldy)
end subroutine    

subroutine mydgemv_a_z_284_(a_t, x, y, lda, ldx, ldy, shift) Bind(C,Name='mydgemv_a_z_284_')
    Import
    Integer(c_int64_t),Value :: lda, ldx, ldy, shift
    Real(c_double),Intent(In) :: a_t(ldx, lda), x(ldx), y(ldy)
end subroutine    

subroutine mydgemv_a_z_286_(a_t, x, y, lda, ldx, ldy, shift) Bind(C,Name='mydgemv_a_z_286_')
    Import
    Integer(c_int64_t),Value :: lda, ldx, ldy, shift
    Real(c_double),Intent(In) :: a_t(ldx, lda), x(ldx), y(ldy)
end subroutine    

subroutine mydgemv_a_z_288_(a_t, x, y, 

In [ ]:
for i in range(4, 33, 1):
    print(f"if (ldx == {i}) mydgemv_c_{i}(a_t, x, y, lda, ldx, ldy);")

In [ ]:
for i in range(1, 33, 1):
    print(f"""
    if (ldx == {i}) {{
        double (*funcPtr)(double[], double[]) = mydgemv_kernel_{i};
    }}""")

In [119]:
1022%8

6

In [120]:
1022 + 8 - 1022%8

1024

In [12]:
with open("./naive_c_implementation_8-128.txt", "w") as f:
    last_idxs = (0, 2)
    for lll in range(4, 128, 2):
        ldx = lll
        declare_x = "double " + ", ".join([f"r{ld:0=2}" for ld in range(ldx)])

        mul_add = [
            f"""
                r{ld+0:0=2} = a_t[{ld}+base_idx];
                r{ld+0:0=2} = x[{ld}]*r{ld+0:0=2};    
            """        
            if ( (ldx % 2 == 1) & (ld==ldx-1) ) else
            f"""
                r{ld+0:0=2} = a_t[{ld}+base_idx];
                r{ld+1:0=2} = a_t[{ld+1}+base_idx]; 
                r{ld+0:0=2} = x[{ld}]*r{ld+0:0=2} + x[{ld+1}]*r{ld+1:0=2};    
            """
            for ld in range(0,ldx,2)
        ]
        mul_add = "".join(mul_add)

        start_step = 4
        accm = ""
        step_size = start_step
        while (step_size < ldx):
            accm += "".join(
                [f"""
                r{ld:0=2} = r{ld:0=2} + r{ld+step_size//2:0=2};
                r{ld+step_size:0=2} = r{ld+step_size:0=2} + r{ld+step_size+step_size//2:0=2};
                """
                if (ld+step_size+step_size//2<ldx) else
                f"""
                r{ld:0=2} = r{ld:0=2} + r{ld+step_size//2:0=2};
                """
                for ld in range(0, ldx, step_size*2)])

            last_idxs = [(ld, ld+step_size) for ld in range(0, ldx, step_size*2)][0]
            step_size *= 2
        accm += f"""
                r{last_idxs[0]:0=2} = r{last_idxs[0]:0=2} + r{last_idxs[1]:0=2};
        """
        # print(accm)

        base = f"""
        void mydgemv_c_{ldx}___(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
            {declare_x};

            int64_t base_idx;
            for (int i=0; i<lda; i++){{
                base_idx = i*ldx;
                {mul_add}
                {accm}
                y[i]=r{last_idxs[0]:0=2};
            }}
        }}
        """
#         print(base)
#         f.write(base)

# Common

In [4]:
import re

def replace_multiple_newlines(text):
    # 3つ以上の連続する改行を2つの改行に置換
    return re.sub(r'\n{2,}', '\n\n', remove_spaces_from_blank_lines(text))

def remove_spaces_from_blank_lines(text):
    # 各行を処理し、スペースのみの行を空行に置き換える
    return re.sub(r'^\s+$', '', text, flags=re.MULTILINE)

# Ver 1, 変数をカラム数文確保しての実装

In [40]:
from copy import deepcopy
def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        return variables[0]

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = {variables_[i]} + {variables_[i+1]};")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)
    
with open(f"../Desktop/FortLearner_latest_20220502/FortLearner/work/small_gemv/small_dgemv_naive_c_implementation_ver1.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    
    for ldx in range(2, 514, 2):

        # Declaration
        declare_x = "double " + ", ".join([f"r{ld}" for ld in range(ldx)])
        variables = [f"r{ld}" for ld in range(ldx)]

        # Initialize
        mul_add = ""
        accm = ""

        # Multiply and Add
        accm_variables = []
        for ld in range(0, len(variables), 2):
            mul_add += f"""
        r{ld+0} = a_t[{ld}+base_idx];
        r{ld+1} = a_t[{ld+1}+base_idx]; 
        r{ld+0} = x[{ld}]*r{ld+0} + x[{ld+1}]*r{ld+1};\n"""
            accm_variables.append(f"r{ld+0}")

        #　Accumulate
        accm_op = []
        accumulator_naive_c(accm_op, accm_variables, indent_level=2)
        accm_op = "\n".join(accm_op)

        # Create Function
        base = f"""
    void mydgemv_ver1_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
    {declare_x};

    int64_t base_idx;
    for (int i=0; i<lda; i++){{
        base_idx = i*ldx + shift;
    {mul_add}
    {accm_op}
        y[i]=r0;
    }}
    }}
        """
        p.write(replace_multiple_newlines(base))

# Ver 2, 変数を最大１６個までに制限する

In [143]:
def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]

def select_usable_var(usable_var, prefix="r", set_second=False):
    var1, var2 = "", ""
    indices = find_true_indices(usable_var, 2)
    
    var1 = f"{prefix}{indices[0]}"
    var2 = f"{prefix}{indices[1]}"
    usable_var[indices[0]] = False
    if set_second: usable_var[indices[1]] = False
    return var1, var2, usable_var


with open(f"../Desktop/FortLearner_latest_20220502/FortLearner/work/small_gemv/small_dgemv_naive_c_implementation_ver2.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    
    for ldx in range(2, 514, 2):
        ldx_max = min([ldx, 16])

        # Declaration
        declare_x = "double " + ", ".join([f"r{ld}" for ld in range(ldx_max)])
        variables = [f"r{ld}" for ld in range(ldx_max)]

        usable_var = [True] * ldx_max

        op_concat = ""

        for ld in range(0, ldx, 2):
            var1, var2, usable_var = select_usable_var(usable_var)
            op_concat += f"""
        {var1} = a_t[{ld}+base_idx];
        {var2} = a_t[{ld+1}+base_idx]; 
        {var1} = x[{ld}]*{var1} + x[{ld+1}]*{var2};\n"""

            if sum(usable_var)<=1:
                accm_op = []
                accm_variables = [f"r{i}" for i in range(len(usable_var)) if not usable_var[i]]

                accumulator_naive_c(accm_op, accm_variables, indent_level=4)
                op_concat += "\n".join(accm_op)
                for var in accm_variables[1:]:
                    idx = int(var[1:])
                    usable_var[idx] = True        
        accm_op = []
        accm_variables = [f"r{i}" for i in range(len(usable_var)) if not usable_var[i]]
        accumulator_naive_c(accm_op, accm_variables, indent_level=2)
        op_concat += "\n".join(accm_op)
        base = f"""
    void mydgemv_ver2_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
    {declare_x};

    int64_t base_idx;
    for (int i=0; i<lda; i++){{
        base_idx = i*ldx + shift;
    {op_concat}
        y[i]=r0;
    }}
    }}
        """

    #     print(op_concat)
        p.write(replace_multiple_newlines(base))
    
    
    
    

NameError: name 'ssysysy' is not defined

In [160]:
def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]



def select_usable_xmm(usable_var):
    count_true = sum(usable_var)
    count_mod2 = (count_true>>1)<<1
    count_half = count_true>>1
    indices = find_true_indices(usable_var, count_mod2)
    
    vars1 = [f"xmm{idx}" for idx in indices[:count_half]]
    vars2 = [f"xmm{idx}" for idx in indices[count_half:]]
    for idx in indices:
        usable_var[idx] = False
    return vars1, vars2, usable_var   

def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        accm_op.append(variables[0])
        return 

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = _mm_add_pd({variables_[i]}, {variables_[i+1]});")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)


with open(f"../Desktop/FortLearner_latest_20220502/FortLearner/work/small_gemv/small_dgemv_naive_c_implementation_ver3.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    p.write("""
double sum_xmm_elements(__m128d xmmX) {
    __m128d xmm_high_low = _mm_add_pd(xmmX, _mm_unpackhi_pd(xmmX, xmmX));

    double result;
    _mm_store_sd(&result, xmm_high_low);

    return result;
}    
    """)

    for ldx in range(2, 514, 2):
        ldx_max = min([ldx, 16])
        declare_xmm = "__m128d " + ", ".join([f"xmm{ld}" for ld in range(ldx_max)])
        variables = [f"xmm{ld}" for ld in range(ldx_max)]

        usable_var = [True] * ldx_max
        
        vars1, vars2, usable_var = select_usable_xmm(usable_var)
    
        load_xmm = ""
        op_concat = ""
        
        if ldx<=1000:
            if ldx <=16:
                for i, var in enumerate(vars2):
                    load_xmm += f"""
        {var} = _mm_loadu_pd(&x[{i*2}]);"""


                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm_loadu_pd(&a_t[{i*2}]);"""

                op_concat += "\n"
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])

                op_concat += f"\n        y[i] = sum_xmm_elements({accm_op[-1]}); a_t += {ldx};"

            else:
                for i, var in enumerate(vars2):
                    op_concat += f"""
            {var} = _mm_loadu_pd(&x[{i*2}]);"""
                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm_loadu_pd(&a_t[{i*2}]);"""
                    
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                for var1, var2 in zip(vars1, vars2):
                    usable_var[int(var1[3:])] = True
                    usable_var[int(var2[3:])] = True
                usable_var[0] = False

                for i in range(16, ldx, 2):
                    var1, var2, usable_var = select_usable_var(usable_var, prefix="xmm", set_second=False)

                    op_concat += f"""
            {var1} = _mm_loadu_pd(&x[{i}]);
            {var2} = _mm_loadu_pd(&a_t[{i}]);
            {var1} = _mm_mul_pd({var1}, {var2});
                    """
                    if sum(usable_var)<=1:
                        accm_op = []
                        accm_variables = [f"xmm{i}" for i in range(len(usable_var)) if not usable_var[i]]

                        accumulator_naive_c(accm_op, accm_variables, indent_level=4)
                        op_concat += "\n".join(accm_op[:-1])
                        for var in accm_variables[1:]:
                            idx = int(var[3:])
                            usable_var[idx] = True        
                accm_op = []
                accm_variables = [f"xmm{i}" for i in range(len(usable_var)) if not usable_var[i]]
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                op_concat += f"\n        y[i] = sum_xmm_elements({accm_op[-1]}); a_t += {ldx};"

            
        base = f"""
void mydgemv_ver3_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
    {declare_xmm};
    {load_xmm}
    
    for (int i=0; i<lda; i++){{
        {op_concat}
        
    }}
}}
        """
        
        p.write(base)
#         print(base)
#         if (ldx>=30): break
#         break

In [175]:
def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]



def select_usable_xmm(usable_var):
    count_true = sum(usable_var)
    count_mod2 = (count_true>>1)<<1
    count_half = count_true>>1
    indices = find_true_indices(usable_var, count_mod2)
    
    vars1 = [f"ymm{idx}" for idx in indices[:count_half]]
    vars2 = [f"ymm{idx}" for idx in indices[count_half:]]
    for idx in indices:
        usable_var[idx] = False
    return vars1, vars2, usable_var   

def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        accm_op.append(variables[0])
        return 

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = _mm256_add_pd({variables_[i]}, {variables_[i+1]});")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)


with open(f"../Desktop/FortLearner_latest_20220502/FortLearner/work/small_gemv/small_dgemv_naive_c_implementation_ver4.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    p.write("""
double sum_ymm_elements(__m256d ymmX) {
    __m128d xmm_upper = _mm256_extractf128_pd(ymmX, 1);
    __m128d xmm_lower = _mm256_extractf128_pd(ymmX, 0);

    __m128d xmm_sum = _mm_add_pd(xmm_upper, xmm_lower);

    __m128d xmm_high_low = _mm_add_pd(xmm_sum, _mm_unpackhi_pd(xmm_sum, xmm_sum));

    double result;
    _mm_store_sd(&result, xmm_high_low);

    return result;
}    
    """)

    for ldx in range(4, 514, 4):
        ldx_max = min([ldx>>1, 16])
        declare_xmm = "__m256d " + ", ".join([f"ymm{ld}" for ld in range(ldx_max)])
        variables = [f"ymm{ld}" for ld in range(ldx_max)]

        usable_var = [True] * ldx_max
        
        vars1, vars2, usable_var = select_usable_xmm(usable_var)
    
        load_xmm = ""
        op_concat = ""
        
        if ldx<=1000:
            if ldx <=32:
                for i, var in enumerate(vars2):
                    load_xmm += f"""
        {var} = _mm256_loadu_pd(&x[{i*4}]);"""


                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm256_loadu_pd(&a_t[{i*4}]);"""

                op_concat += "\n"
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm256_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])

                op_concat += f"\n        y[i] = sum_ymm_elements({accm_op[-1]}); a_t += {ldx};"

            else:
                for i, var in enumerate(vars2):
                    op_concat += f"""
            {var} = _mm256_loadu_pd(&x[{i*4}]);"""
                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm256_loadu_pd(&a_t[{i*4}]);"""
                    
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm256_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                for var1, var2 in zip(vars1, vars2):
                    usable_var[int(var1[3:])] = True
                    usable_var[int(var2[3:])] = True
                usable_var[0] = False

                for i in range(32, ldx, 4):
                    var1, var2, usable_var = select_usable_var(usable_var, prefix="ymm", set_second=False)

                    op_concat += f"""
            {var1} = _mm256_loadu_pd(&x[{i}]);
            {var2} = _mm256_loadu_pd(&a_t[{i}]);
            {var1} = _mm256_mul_pd({var1}, {var2});
                    """
                    if sum(usable_var)<=1:
                        accm_op = []
                        accm_variables = [f"ymm{i}" for i in range(len(usable_var)) if not usable_var[i]]

                        accumulator_naive_c(accm_op, accm_variables, indent_level=4)
                        op_concat += "\n".join(accm_op[:-1])
                        for var in accm_variables[1:]:
                            idx = int(var[3:])
                            usable_var[idx] = True        
                accm_op = []
                accm_variables = [f"ymm{i}" for i in range(len(usable_var)) if not usable_var[i]]
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                op_concat += f"\n        y[i] = sum_ymm_elements({accm_op[-1]}); a_t += {ldx};"

            
        base = f"""
void mydgemv_ver4_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
    {declare_xmm};
    {load_xmm}
    
    for (int i=0; i<lda; i++){{
        {op_concat}
        
    }}
}}
        """
        
        p.write(base)
#         print(base)
#         if (ldx>=30): break
#         break

In [185]:
def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]



def select_usable_xmm(usable_var):
    count_true = sum(usable_var)
    count_mod2 = (count_true>>1)<<1
    count_half = count_true>>1
    indices = find_true_indices(usable_var, count_mod2)
    
    vars1 = [f"zmm{idx}" for idx in indices[:count_half]]
    vars2 = [f"zmm{idx}" for idx in indices[count_half:]]
    for idx in indices:
        usable_var[idx] = False
    return vars1, vars2, usable_var   

def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        accm_op.append(variables[0])
        return 

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = _mm512_add_pd({variables_[i]}, {variables_[i+1]});")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)


with open(f"../Desktop/FortLearner_latest_20220502/FortLearner/work/small_gemv/small_dgemv_naive_c_implementation_ver5.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    p.write("""
double sum_zmm_elements(__m512d zmmX) {
    __m256d ymm_upper = _mm512_extractf64x4_pd(zmmX, 1);
    __m256d ymm_lower = _mm512_extractf64x4_pd(zmmX, 0);

    __m256d ymm_sum = _mm256_add_pd(ymm_upper, ymm_lower);

    __m128d xmm_sum = _mm_add_pd(_mm256_extractf128_pd(ymm_sum, 1), _mm256_extractf128_pd(ymm_sum, 0));

    __m128d xmm_high_low = _mm_add_pd(xmm_sum, _mm_unpackhi_pd(xmm_sum, xmm_sum));

    double result;
    _mm_store_sd(&result, xmm_high_low);

    return result;
}
    """)

    for ldx in range(8, 514, 8):
        ldx_max = min([ldx>>2, 16])
        declare_xmm = "__m512d " + ", ".join([f"zmm{ld}" for ld in range(ldx_max)])
        variables = [f"zmm{ld}" for ld in range(ldx_max)]

        usable_var = [True] * ldx_max
        
        vars1, vars2, usable_var = select_usable_xmm(usable_var)
    
        load_xmm = ""
        op_concat = ""
        
        if ldx<=1000:
            if ldx <=64:
                for i, var in enumerate(vars2):
                    load_xmm += f"""
        {var} = _mm512_loadu_pd(&x[{i*8}]);"""


                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm512_loadu_pd(&a_t[{i*8}]);"""

                op_concat += "\n"
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm512_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])

                op_concat += f"\n        y[i] = sum_zmm_elements({accm_op[-1]}); a_t += {ldx};"

            else:
                for i, var in enumerate(vars2):
                    op_concat += f"""
            {var} = _mm512_loadu_pd(&x[{i*8}]);"""
                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm512_loadu_pd(&a_t[{i*8}]);"""
                    
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm512_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                for var1, var2 in zip(vars1, vars2):
                    usable_var[int(var1[3:])] = True
                    usable_var[int(var2[3:])] = True
                usable_var[0] = False

                for i in range(64, ldx, 8):
                    var1, var2, usable_var = select_usable_var(usable_var, prefix="zmm", set_second=False)

                    op_concat += f"""
            {var1} = _mm512_loadu_pd(&x[{i}]);
            {var2} = _mm512_loadu_pd(&a_t[{i}]);
            {var1} = _mm512_mul_pd({var1}, {var2});
                    """
                    if sum(usable_var)<=1:
                        accm_op = []
                        accm_variables = [f"zmm{i}" for i in range(len(usable_var)) if not usable_var[i]]

                        accumulator_naive_c(accm_op, accm_variables, indent_level=4)
                        op_concat += "\n".join(accm_op[:-1])
                        for var in accm_variables[1:]:
                            idx = int(var[3:])
                            usable_var[idx] = True        
                accm_op = []
                accm_variables = [f"zmm{i}" for i in range(len(usable_var)) if not usable_var[i]]
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                op_concat += f"\n        y[i] = sum_zmm_elements({accm_op[-1]}); a_t += {ldx};"

            
        base = f"""
void mydgemv_ver5_c_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy, int64_t shift){{
    {declare_xmm};
    {load_xmm}
    
    for (int i=0; i<lda; i++){{
        {op_concat}
        
    }}
}}
        """
        
        p.write(base)
#         print(base)
#         if (ldx>=30): break
#         break

In [154]:
for i in range(2, 514, 2):
#     print(f"	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver1/small_dgemv_naive_c_implementation_d{i}_ver1.c -o ./mod/small_dgemv_naive_c_implementation_d{i}_ver1.out")
    print(f"""
        subroutine mydgemv_ver3_c_{i}(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_ver3_c_{i}')
            import
            integer(c_int64_t), value :: lda, ldx, ldy, shift
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_ver3_c_{i}
          """)
#     print(f"if (ldx == {i}) sub_ptr => mydgemv_ver1_c_{i}")


        subroutine mydgemv_ver3_c_2(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_ver3_c_2')
            import
            integer(c_int64_t), value :: lda, ldx, ldy, shift
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_ver3_c_2
          

        subroutine mydgemv_ver3_c_4(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_ver3_c_4')
            import
            integer(c_int64_t), value :: lda, ldx, ldy, shift
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_ver3_c_4
          

        subroutine mydgemv_ver3_c_6(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_ver3_c_6')
            import
            integer(c_int64_t), value :: lda, ldx, ldy, shift
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_ver3_c_6
          

        subroutine mydgemv_ver3_c_8(a_t, x, y, lda, ldx, ldy, shift) bind(C, n

In [180]:
for i in range(4, 514, 4):
#     print(f"	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver1/small_dgemv_naive_c_implementation_d{i}_ver1.c -o ./mod/small_dgemv_naive_c_implementation_d{i}_ver1.out")
#     print(f"""
#         subroutine mydgemv_ver4_c_{i}(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_ver4_c_{i}')
#             import
#             integer(c_int64_t), value :: lda, ldx, ldy, shift
#             real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
#         end subroutine mydgemv_ver_c_{i}
#           """)
    print(f"if (ldx == {i}) sub_ptr => mydgemv_ver4_c_{i}")

if (ldx == 4) sub_ptr => mydgemv_ver4_c_4
if (ldx == 8) sub_ptr => mydgemv_ver4_c_8
if (ldx == 12) sub_ptr => mydgemv_ver4_c_12
if (ldx == 16) sub_ptr => mydgemv_ver4_c_16
if (ldx == 20) sub_ptr => mydgemv_ver4_c_20
if (ldx == 24) sub_ptr => mydgemv_ver4_c_24
if (ldx == 28) sub_ptr => mydgemv_ver4_c_28
if (ldx == 32) sub_ptr => mydgemv_ver4_c_32
if (ldx == 36) sub_ptr => mydgemv_ver4_c_36
if (ldx == 40) sub_ptr => mydgemv_ver4_c_40
if (ldx == 44) sub_ptr => mydgemv_ver4_c_44
if (ldx == 48) sub_ptr => mydgemv_ver4_c_48
if (ldx == 52) sub_ptr => mydgemv_ver4_c_52
if (ldx == 56) sub_ptr => mydgemv_ver4_c_56
if (ldx == 60) sub_ptr => mydgemv_ver4_c_60
if (ldx == 64) sub_ptr => mydgemv_ver4_c_64
if (ldx == 68) sub_ptr => mydgemv_ver4_c_68
if (ldx == 72) sub_ptr => mydgemv_ver4_c_72
if (ldx == 76) sub_ptr => mydgemv_ver4_c_76
if (ldx == 80) sub_ptr => mydgemv_ver4_c_80
if (ldx == 84) sub_ptr => mydgemv_ver4_c_84
if (ldx == 88) sub_ptr => mydgemv_ver4_c_88
if (ldx == 92) sub_ptr => mydgemv_ve

In [181]:
for i in range(8, 514, 8):
#     print(f"	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver1/small_dgemv_naive_c_implementation_d{i}_ver1.c -o ./mod/small_dgemv_naive_c_implementation_d{i}_ver1.out")
#     print(f"""
#         subroutine mydgemv_ver5_c_{i}(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_ver5_c_{i}')
#             import
#             integer(c_int64_t), value :: lda, ldx, ldy, shift
#             real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
#         end subroutine mydgemv_ver5_c_{i}
#           """)
    print(f"if (ldx == {i}) sub_ptr => mydgemv_ver5_c_{i}")

if (ldx == 8) sub_ptr => mydgemv_ver5_c_8
if (ldx == 16) sub_ptr => mydgemv_ver5_c_16
if (ldx == 24) sub_ptr => mydgemv_ver5_c_24
if (ldx == 32) sub_ptr => mydgemv_ver5_c_32
if (ldx == 40) sub_ptr => mydgemv_ver5_c_40
if (ldx == 48) sub_ptr => mydgemv_ver5_c_48
if (ldx == 56) sub_ptr => mydgemv_ver5_c_56
if (ldx == 64) sub_ptr => mydgemv_ver5_c_64
if (ldx == 72) sub_ptr => mydgemv_ver5_c_72
if (ldx == 80) sub_ptr => mydgemv_ver5_c_80
if (ldx == 88) sub_ptr => mydgemv_ver5_c_88
if (ldx == 96) sub_ptr => mydgemv_ver5_c_96
if (ldx == 104) sub_ptr => mydgemv_ver5_c_104
if (ldx == 112) sub_ptr => mydgemv_ver5_c_112
if (ldx == 120) sub_ptr => mydgemv_ver5_c_120
if (ldx == 128) sub_ptr => mydgemv_ver5_c_128
if (ldx == 136) sub_ptr => mydgemv_ver5_c_136
if (ldx == 144) sub_ptr => mydgemv_ver5_c_144
if (ldx == 152) sub_ptr => mydgemv_ver5_c_152
if (ldx == 160) sub_ptr => mydgemv_ver5_c_160
if (ldx == 168) sub_ptr => mydgemv_ver5_c_168
if (ldx == 176) sub_ptr => mydgemv_ver5_c_176
if (ldx == 184

In [33]:
for i in range(2, 514, 2):
    print(f"	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver2/small_dgemv_naive_c_implementation_d{i}_ver2.c -o ./mod/small_dgemv_naive_c_implementation_d{i}_ver2.out")

	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver2/small_dgemv_naive_c_implementation_d2_ver2.c -o ./mod/small_dgemv_naive_c_implementation_d2_ver2.out
	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver2/small_dgemv_naive_c_implementation_d4_ver2.c -o ./mod/small_dgemv_naive_c_implementation_d4_ver2.out
	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver2/small_dgemv_naive_c_implementation_d6_ver2.c -o ./mod/small_dgemv_naive_c_implementation_d6_ver2.out
	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver2/small_dgemv_naive_c_implementation_d8_ver2.c -o ./mod/small_dgemv_naive_c_implementation_d8_ver2.out
	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver2/small_dgemv_naive_c_implementation_d10_ver2.c -o ./mod/small_dgemv_naive_c_implementation_d10_ver2.out
	$(compiler_clang)  $(mod) $(opt) $(option) -c    ./ver2/small_dgemv_naive_c_implementation_d12_ver2.c -o ./mod/small_dgemv_naive_c_implementation_d12_ver2.out
	$(compiler_clang)  $(mod) $(opt) $(option) -c  

In [62]:
7, 7>>1, (7>>1)<<1

(7, 3, 6)